In [ ]:
pip install --upgrade transformers accelerate

In [2]:
from transformers import BitsAndBytesConfig, AutoTokenizer, AutoModelForCausalLM
from transformers.cache_utils import DynamicCache
import torch

c:\Source\Finance_Bench\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
            model_name,
            torch_dtype=torch.float16,
            device_map="auto",
)

OSError: You are trying to access a gated repo.
Make sure to have access to it at https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct.
401 Client Error. (Request ID: Root=1-678d11f8-75dc3e18577d686943fcbe66;0c94335b-c5dd-4ebf-a195-86e5dc115f69)

Cannot access gated repo for url https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct/resolve/main/config.json.
Access to model meta-llama/Llama-3.2-1B-Instruct is restricted. You must have access to it and be authenticated to access it. Please log in.

In [5]:
torch.serialization.add_safe_globals([DynamicCache])
torch.serialization.add_safe_globals([set])

In [10]:
def write_kv_cache(kv: DynamicCache, path: str):
    """
    Write the KV Cache to a file.
    """
    torch.save(kv, path)


def clean_up(kv: DynamicCache, origin_len: int):
    """
    Truncate the KV Cache to the original length.
    """
    for i in range(len(kv.key_cache)):
        kv.key_cache[i] = kv.key_cache[i][:, :, :origin_len, :]
        kv.value_cache[i] = kv.value_cache[i][:, :, :origin_len, :]


def read_kv_cache(path: str) -> DynamicCache:
    """
    Read the KV Cache from a file.
    """
    kv = torch.load(path, weights_only=True)
    return kv

In [6]:
def preprocess_knowledge(
    model,
    tokenizer,
    prompt: str,
) -> DynamicCache:
    """
    Prepare knowledge kv cache for CAG.
    Args:
        model: HuggingFace model with automatic device mapping
        tokenizer: HuggingFace tokenizer
        prompt: The knowledge to preprocess, which is basically a prompt

    Returns:
        DynamicCache: KV Cache
    """
    embed_device = model.model.embed_tokens.weight.device
    input_ids = tokenizer.encode(prompt, return_tensors="pt").to(embed_device)
    past_key_values = DynamicCache()
    with torch.no_grad():
        outputs = model(
            input_ids=input_ids,
            past_key_values=past_key_values,
            use_cache=True,
            output_attentions=False,
            output_hidden_states=False
        )
    return outputs.past_key_values

In [4]:
def prepare_kvcache(documents, filepath: str = "./data_cache/cache_knowledges.pt", answer_instruction: str = None):
    # Prepare the knowledges kvcache

    if answer_instruction is None:
        answer_instruction = "Answer the question with a super short answer."
    knowledges = f"""
    <|begin_of_text|>
    <|start_header_id|>system<|end_header_id|>
    You are an assistant for giving short answers based on given context.<|eot_id|>
    <|start_header_id|>user<|end_header_id|>
    Context information is bellow.
    ------------------------------------------------
    {documents}
    ------------------------------------------------
    {answer_instruction}
    Question:
    """
    # Get the knowledge cache
    t1 = time()
    kv = preprocess_knowledge(model, tokenizer, knowledges)
    print("kvlen: ", kv.key_cache[0].shape[-2])
    write_kv_cache(kv, filepath)
    t2 = time()
    return kv, t2 - t1

In [9]:
Text = """
Building a Balanced Growth Stock Portfolio in the Indian Market
Investing in growth stocks can be an exciting way to build wealth over the long term. Growth stocks are those issued by companies that are expected to grow at an above-average rate compared to other companies in the market. However, investing in growth stocks can also be risky, as these companies are often more volatile than more established companies.

To mitigate risk and maximize returns, it's essential to create a balanced portfolio of growth stocks across different sectors in the Indian market. This approach helps to spread risk and capture growth opportunities across various segments of the economy. Building a successful growth stock portfolio requires careful consideration of risk and potential rewards.   

Research Methodology
To ensure a comprehensive analysis of the Indian stock market and identify potential growth stocks, the following research steps were conducted:

Gathering Information on Growth Stock Portfolios: Articles and research reports on creating a balanced portfolio of growth stocks were reviewed to understand the underlying principles and best practices.
Analyzing the Indian Market: Information on the different sectors in the Indian market, their growth potential, and the associated risks was gathered.
Identifying Top Growth Stocks: Top-performing growth stocks in each sector were identified based on factors such as market capitalization, financial performance, and growth prospects.
Assessing Sector-Specific Risks: Information on the risks associated with investing in each sector was collected to understand the potential challenges and develop mitigation strategies.
Developing Sector Allocation Strategies: Information on how to allocate a portfolio across different sectors was reviewed, considering factors such as risk tolerance, investment goals, and time horizon.
Understanding the Indian Stock Market
The Indian stock market is one of the largest and fastest-growing in the world. It is home to a wide range of companies across various sectors, including information technology (IT), healthcare, fast-moving consumer goods (FMCG), renewable energy, and infrastructure. These sectors are interconnected, and growth in one can often influence others. For example, the growth of the IT sector can drive demand for healthcare services and consumer goods.   

To build a balanced growth stock portfolio, it's crucial to understand the different sectors in the Indian market and their growth potential. Here are some of the key sectors to consider:

IT
The Indian IT sector is a global leader, with companies like Tata Consultancy Services (TCS), Infosys, and HCL Technologies providing IT services and consulting to clients worldwide. This sector is expected to continue its growth trajectory, driven by the increasing adoption of digital technologies and cloud computing.   

Healthcare
The healthcare sector in India is experiencing rapid growth, driven by factors such as an aging population, rising incomes, and increasing awareness of health and wellness. This sector offers a wide range of investment opportunities, from pharmaceutical companies and hospitals to diagnostic centers.  The rise of chronic diseases and the increasing use of digital health platforms are also contributing to the sector's growth.   

FMCG
The FMCG sector is a defensive sector that is relatively less affected by economic downturns. This sector includes companies that produce a wide range of consumer goods, including food, beverages, personal care products, over-the-counter drugs, consumer electronics, and stationery. The Indian FMCG sector is expected to grow steadily, driven by rising disposable incomes and changing consumer preferences.   

Renewable Energy
India is making significant strides in renewable energy, with the government setting ambitious targets for solar and wind power generation. This sector offers attractive investment opportunities, with companies like Adani Green Energy and Tata Power leading the way.   

Infrastructure
India's infrastructure sector is undergoing a massive transformation, with the government investing heavily in roads, railways, and other infrastructure projects. This sector offers long-term growth potential, with companies like Larsen & Toubro (L&T) and IRB Infrastructure Developers well-positioned to benefit.   

Industry Associations
In addition to understanding the individual sectors, it's important to be aware of the role of industry associations in the Indian market. These associations play a crucial role in shaping industry trends, advocating for policy changes, and promoting best practices. Some of the key industry associations in India include:

Sector	Association
Cement	Cement Manufacturers Association
Automobiles	Society of Indian...source

Export to Sheets
These associations provide valuable insights into the dynamics of their respective sectors and can be a useful resource for investors.

It's also important to note the historical relationship between equities and bonds in the Indian market. Studies have shown that their correlation is mostly negative when valuations for both asset classes are not expensive. This negative correlation can provide diversification benefits, as bonds tend to perform well when equities decline, and vice versa.   

Identifying Top Growth Stocks in Each Sector
Once you have a good understanding of the different sectors in the Indian market, you can start identifying the top growth stocks in each sector. When selecting stocks, it's generally recommended to favor stock mutual funds or ETFs over individual company stocks. Funds provide instant diversification and professional management, while individual stocks carry higher risk.   

Here are some of the top-performing growth stocks in India across different sectors:

IT Sector
The Indian IT sector is characterized by its strong growth potential, driven by factors such as the increasing adoption of digital technologies, cloud computing, and artificial intelligence. However, investors should also consider factors such as company size, innovation, competition, industry trends, and economic conditions before investing in IT stocks.   

Company	Market Cap (₹ Cr)	Close Price (₹)
Tata Consultancy Services	15,44,000	4,430.05
Infosys	7,77,000	1,990.25
HCL Technologies	4,87,000	1,958.65
Wipro	2,82,000	307.55
LTIMindtree	1,85,000	6,732.00
Tech Mahindra	1,53,000	1,772.25

Export to Sheets
Healthcare Sector
The healthcare sector in India is poised for continued growth, driven by factors such as an aging population, rising incomes, and increasing healthcare expenditure. Companies like Apollo Hospitals and Fortis Healthcare are well-positioned to benefit from this growth.   

Company	Market Cap (₹ Cr)	Close Price (₹)
Max Healthcare Institute	1,09,671.08	1,128.15
Apollo Hospitals Enterprise	1,04,910.32	7,296.35
Fortis Healthcare	54,345.66	719.85
Global Health	28,967.15	1,078.50
Narayana Hrudayalaya	25,850.55	1,272.80

Export to Sheets
Pharmaceutical Sector
The pharmaceutical sector in India is a global player, with companies like Sun Pharmaceuticals and Dr Reddy's Laboratories exporting their products worldwide. When evaluating pharmaceutical stocks, it's essential to consider the company's research and development (R&D) pipeline, as this is a key indicator of its future growth potential.   

Company	LTP	Market Cap (Cr.)
Sun Pharmaceuticals Industries	1,763.30	₹ 423,074.74
Dr Reddys Laboratories	1,302.75	₹ 108,704.58
Cipla	1,443.50	₹ 116,579.09
Lupin	2,098.90	₹ 95,759.07
Aurobindo Pharma	1,156.30	₹ 67,752.08

Export to Sheets
Renewable Energy Sector
The renewable energy sector in India is experiencing rapid growth, driven by government initiatives and increasing awareness of the need for clean energy. Companies like Adani Green Energy and JSW Energy are at the forefront of this growth.

Company	LTP (₹)	Market Cap (Cr)
Sterling and Wilson Renewable Energy	457.65	10,686
Energy Development Company	26.90	128
Adani Green Energy	1,070.25	1,69,531
Karma Energy	66.09	76
SJVN	97.26	38,221

Export to Sheets
Solar Energy Sector
The solar energy sector is a key component of India's renewable energy push. Companies like Tata Power Company and Suzlon Energy are major players in this sector.

Company Name	Market Cap (Cr)	Close Price (Rs.)
Adani Green Energy	1,63,947	1,070
Tata Power Company	1,16,790	368.9
JSW Energy	98,102	570.7
Suzlon Energy	78,189	56.88
SJVN	37,235	97.20

Export to Sheets
FMCG Sector
The FMCG sector in India is characterized by strong brand loyalty and a large consumer base. Companies like Hindustan Unilever and ITC Limited have a dominant presence in this sector.

Stock Name	Market Cap	Closing Price
ITC Limited	5,933,688,462,747	475
Hindustan Unilever Limited (HUL)	5,493,925,700,874	2,462
Varun Beverages	2,121,559,456,556	631
Nestle India	2,073,970,554,106	2,236
Britannia Industries	1,132,633,809,478	4,924

Export to Sheets
Infrastructure Sector
The infrastructure sector in India is undergoing a major transformation, with significant investments in roads, railways, and other infrastructure projects. Companies like Larsen & Toubro and KEC International are well-positioned to benefit from this growth.

| Company Name | CMP (13th Jan 2025) |
|---|
| Larsen & Toubro Ltd. | 3528.25 |
| Rail Vikas Nigam Ltd. | 373.05 |
| IRB Infrastructure Development | 51.91 |
| KEC International Ltd. | 945.00 |
| NBCC (India) | 82.35 |    

Cement Sector
The cement sector in India is expected to grow steadily, driven by increasing demand from the housing and infrastructure sectors. Companies like Ultratech Cement and Shree Cement are major players in this sector.

Stock Name	Market Cap	P/E Ratio
Ultratech Cement	3400813100000	51.95
Shree Cement	980745144224	56.72
Ambuja Cement	1405211101000	43.65
ACC Cement	417882865329	20.52
Dalmia Bharat Cement	362750903400	47.42

Export to Sheets
Financial Services Sector
The financial services sector in India is a key driver of economic growth. Companies like HDFC Bank and ICICI Bank are leading players in this sector.

Name	CMP Rs.	P/E	Mar Cap Rs.Cr.
HDFC Bank	1652.05	18.26	1263539.97
ICICI Bank	1249.10	18.62	881874.56
St Bk of India	766.30	8.93	683893.63
Bajaj Finance	7259.75	29.23	449375.96
Kotak Mah. Bank	1805.55	18.67	358974.70

Export to Sheets
Banking Sector
The banking sector in India is undergoing a period of consolidation and digital transformation. Companies like IDBI Bank and Indian Bank are among the key players in this sector.

Company	LTP (₹)	Market Cap (Cr)
IDBI Bank	79.48	85,460
Punjab & Sind Bank	49.73	33,706
Jana Small Finance Bank	393.70	4,124
Indian Bank	527.65	71,073
Union Bank of India	110.33	84,222

Export to Sheets
It's important to conduct thorough research and due diligence before investing in any of these stocks. Consider factors such as the company's financial performance, growth prospects, competitive landscape, and management quality.

Managing Risk
While growth stocks offer the potential for high returns, they also come with inherent risks. Here are some of the risks associated with investing in growth stocks in the Indian market:

Market Volatility
Growth stocks tend to be more volatile than value stocks, meaning their prices can fluctuate significantly in the short term. This volatility can be driven by factors such as market sentiment, economic conditions, and company-specific news.   

Regulatory Changes
Changes in government policies and regulations can impact the growth prospects of certain sectors and companies. For example, changes in environmental regulations can affect the renewable energy sector, while changes in healthcare policy can impact pharmaceutical companies.

Competition
The Indian market is highly competitive, and companies need to constantly innovate and adapt to stay ahead of the curve. This competition can put pressure on profit margins and make it challenging for companies to sustain high growth rates.

Technological Disruption
Rapid technological advancements can disrupt existing business models and impact the growth of certain companies. For example, the rise of e-commerce has disrupted traditional retail businesses, while the growth of digital technologies has impacted the IT sector.

Sector-Specific Risks
In addition to the general risks mentioned above, each sector also faces its own unique set of risks. For example, renewable energy companies are particularly vulnerable to supply chain disruptions and fluctuating market prices for renewable energy technology. Consumer staples companies face risks such as changing consumer trends, product recalls, and ineffective marketing.   

Other Risks
Investors should also be aware of other risks associated with financial markets, such as counterparty risk, interest rate risk, and default risk. Counterparty risk is the risk that a party to a financial contract will not fulfill its obligations. Interest rate risk is the risk that changes in interest rates will affect the value of investments. Default risk is the risk that a borrower will not repay a loan.   

When investing in a global infrastructure strategy, it's important to consider currency volatility. Changes in exchange rates can affect the returns on investments in foreign currencies.   

To manage these risks, it's essential to:

Diversify your portfolio: Invest in growth stocks across different sectors to spread risk.
Conduct thorough research: Before investing in any stock, carefully analyze the company's financials, growth prospects, and competitive landscape.
Invest for the long term: Growth stocks are best suited for long-term investors who can withstand short-term volatility.
Monitor your portfolio regularly: Keep track of your investments and make adjustments as needed.
Be wary of highly acquisitive companies: Scrutinize companies that pursue earnings growth through acquisitions, especially if they have a lot of debt or concentrated sales in a small group of products.   
Consider the broader market context: Be aware of the potential impact of market cycles, economic outlook, and investor preferences on your investment decisions.
Investment Strategies for Growth Stocks
In addition to managing risk, investors should also consider specific investment strategies for growth stocks. These strategies can help to maximize returns and achieve investment goals.

Power of Compound Interest
One of the key advantages of investing in growth stocks is the power of compound interest. Compound interest is the interest earned on both the principal amount and the accumulated interest. Over time, compound interest can significantly increase the value of investments.   

Automating Investments
Another effective strategy is to automate investments. This involves setting up regular contributions to your investment account, such as through a systematic investment plan (SIP). Automating investments helps to ensure that you consistently invest in the market, regardless of market fluctuations.   

Value Investing
While the focus of this article is on growth stocks, it's also worth considering value stocks as part of a balanced portfolio. Value stocks are those that are trading below their intrinsic value. One of the most effective valuation metrics for value stocks is the Price to Cash Flow (P/CF) ratio, which measures the market price of a stock relative to the cash flow the company generates on a per-share basis.   

Allocating Your Portfolio Across Sectors
The ideal sector allocation for your growth stock portfolio will depend on your individual risk tolerance, investment goals, and time horizon. It's also important to consider your time horizon, risk tolerance, and financial goals when deciding how much of your portfolio to allocate toward consumer staples. However, as a general guideline, you may consider the following allocation:   

Sector	Suggested Allocation
IT	20-30%
Healthcare	15-25%
FMCG	10-15%
Renewable Energy	10-15%
Infrastructure	10-15%
Financial Services	10-15%

Export to Sheets
This is just a suggested allocation, and you may need to adjust it based on your specific circumstances. For example, if you have a higher risk tolerance, you may allocate a larger portion of your portfolio to high-growth sectors like IT and renewable energy. On the other hand, if you are closer to retirement, you may want to reduce your exposure to these sectors and increase your allocation to more defensive sectors like FMCG.   

It's important to remember that sector allocation is not a one-time decision. You should regularly review and rebalance your portfolio to ensure that it continues to align with your investment goals and risk tolerance.

Portfolio Rebalancing
Rebalancing your portfolio is an important aspect of maintaining your desired asset allocation. It involves adjusting your holdings to return to your target mix, ensuring that no single asset class becomes too dominant or underrepresented as market conditions change. This can help to prevent style drift, which is the tendency for a portfolio to deviate from its intended style over time.   

For example, if your target allocation for the IT sector is 25%, but the sector has performed well and now represents 35% of your portfolio, you would sell some of your IT holdings and invest in other sectors to bring the allocation back to 25%.

Conclusion
Building a balanced growth stock portfolio in the Indian market requires careful planning, research, and risk management. By understanding the different sectors, identifying top growth stocks, and allocating your portfolio strategically, you can position yourself to capture the growth opportunities offered by the Indian stock market while mitigating potential risks. Remember to maintain a long-term perspective and a disciplined approach to investing, and regularly rebalance your portfolio to ensure that it continues to meet your investment goals.   

Start building your balanced growth stock portfolio today and take advantage of the exciting opportunities offered by the Indian stock market!

"""

In [ ]:
from time import time
knowledge_cache, prepare_time = prepare_kvcache(Text,"./cache.pt")

In [44]:
question = "Summarize the document like Paul Graham"

In [45]:
prompt = f"""
    {question}<|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """

In [46]:
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(model.device)

In [51]:
def generate(
    model,
    input_ids: torch.Tensor,
    past_key_values,
    max_new_tokens: int = 3000
) -> torch.Tensor:
    """
    Generate text with greedy decoding.

    Args:
        model: HuggingFace model with automatic device mapping
        input_ids: Input token ids
        past_key_values: KV Cache for knowledge
        max_new_tokens: Maximum new tokens to generate
    """

    embed_device = model.model.embed_tokens.weight.device

    origin_ids = input_ids
    input_ids = input_ids.to(embed_device)

    output_ids = input_ids.clone()
    next_token = input_ids

    with torch.no_grad():
        for _ in range(max_new_tokens):
            outputs = model(
                input_ids=next_token, 
                past_key_values=past_key_values,
                use_cache=True
            )
            next_token_logits = outputs.logits[:, -1, :]
            next_token = next_token_logits.argmax(dim=-1).unsqueeze(-1)
            next_token = next_token.to(embed_device)

            past_key_values = outputs.past_key_values

            output_ids = torch.cat([output_ids, next_token], dim=1)

            if next_token.item() in model.config.eos_token_id:
                break
    return output_ids[:, origin_ids.shape[-1]:]

In [55]:
output = generate(model, input_ids, knowledge_cache)

In [56]:
generated_text = tokenizer.decode(output[0], skip_special_tokens=True, temperature=None)

In [ ]:
print(generated_text)